In [7]:
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split,StratifiedKFold
import warnings

rcParams['figure.figsize'] = (16, 8)
pd.options.display.max_columns = 100

In [8]:
#-*- coding: utf-8 -*-

import sys
sys.stdin.encoding

'cp949'

In [12]:
train = pd.read_csv(r"train")
test = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\test.csv")
display(train.head(), test.head())

FileNotFoundError: [Errno 2] File train does not exist: 'train'

EDA

In [ ]:
train.describe()

In [ ]:
sns.heatmap(train.corr(), annot = True, annot_kws = {"size":11}, linewidths = 1)

전처리

In [ ]:
# 동시에 전처리하기 위해 training data와 test data를 concatenate

df = pd.concat([train,test])
df.head()

In [ ]:
# Machine Learning Applied to Star–Galaxy–QSO Classification and Stellar Effective Temperature Regression 논문 참조 파생변수 생성

df["u-g"] = df["u"]-df["g"]
df["g-r"] = df["g"]-df["r"]
df["r-i"] = df["r"]-df["i"]
df["i-z"] = df["i"]-df["z"]

df["d1diff"] = df["dered_u"]-df["dered_g"]
df["d2diff"] = df["dered_g"]-df["dered_r"]
df["d3diff"] = df["dered_r"]-df["dered_i"]
df["d4diff"] = df["dered_i"]-df["dered_z"]

df["minusOD"] = df["nObserve"] - df["nDetect"]
df["plusOD"] = df["nObserve"] + df["nDetect"]

In [ ]:
# EDA 이후 불필요 칼럼, 상관관계, 다중공선성 등...을 고려하여 일부 피쳐 제거

df2 = df.drop(["id","class","airmass_u","airmass_g","airmass_r","airmass_i","airmass_z"], axis=1)
df2.head()

In [ ]:
# df를 train과 test의 크기만큼 다시 분리

train2 = df2[:len(train)]
test2 = df2[len(train):]

RandomForest모델 학습

In [ ]:
clf = RandomForestClassifier(n_estimators=400,
                             n_jobs=-1,
                             verbose = 1)

clf.fit(train2, train["class"])
rf_result = clf.predict_proba(test2)


Xgboost모델 학습

In [ ]:
# StratifiedKFold를 사용하여 교차검증

sfk = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
xgb_result = 0

for i,j in sfk.split(train2,train["class"]):
    x_train = train2.iloc[i]
    x_valid = train2.iloc[j]
    y_train = train["class"].iloc[i]
    y_valid = train["class"].iloc[j]

    xgb = XGBClassifier(objective='multiclass',
                         n_estimators = 500, 
                         max_depth = 9,
                         subsample = 1,
                         colsample_bytree = 1,
                         nthread= -1, 
                         )
    xgb.fit(x_train, y_train, eval_set = [(x_valid, y_valid)], early_stopping_rounds = 70)
    xgb_result += xgb.predict_proba(test2)/10


일정 비율 앙상블

In [ ]:
result = rf_result*0.4 + xgb_result*0.6
final = pd.DataFrame(result)

In [ ]:
sub = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\sample_submission.csv")
sub["class"] = final.argmax(1)
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)